In [2]:
import pandas as pd
import numpy as np
import matplotlib
import json
from pandas import json_normalize
from IPython.display import display

In [3]:
data = pd.read_csv('../../data/healthtap_short_sample.csv', index_col=0)

In [4]:
qa_data = data[['question', 'answer']]

In [5]:
qa_data.head(20)

,question,answer
1354503,Why my inside stomach and sides itch feelling ...,See your doctor. I recommend going to your doc...
2181719,Is glaucoma common in young women?,No. Glaucoma is generally a disease of aging. ...
1903046,I have a 2 year old daughter who was diagnosed...,Ventriculomegaly without hydrocephalus is asso...
132377,Can sex cause cold sores?,Yes - oral sex. One of the most common sexuall...
367237,My brain MRI showed multiple T2/flair hyperint...,yes I would consult a neurologist.
1444266,Is there a back surgery where you can cut off ...,Decompressive laminectomies can remove bony co...
2005266,I have had ongoing ovarian cysts that rupture....,No. No clear association.
1720835,Does subungual melanoma have to be a strait l...,Any shape.
2221199,Having extra beats and spells of hr 130-140. M...,Best to discuss with your doctor to see if som...
1989492,Can you inject humira (adalimumab) anywhere on...,"If the shots are painful, massage the skin of ..."


In [6]:
import re

def phrase_expanding(text):
    text = re.sub(r"(\b)([Ii])'m", r"\1\2 am", text)
    text = re.sub(r"(\b)([Tt]hey|[Ww]e|[Ww]hat|[Ww]ho|[Yy]ou)'re", r"\1\2 are", text)
    text = re.sub(r"(\b)([Ll]et)'s", r"\1\2 us", text)
    text = re.sub(r"(\b)([Hh]e|[Ii]|[Ss]he|[Tt]hey|[Ww]e|[Ww]hat|[Ww]ho|[Yy]ou)'ll", r"\1\2 will", text)
    text = re.sub(r"(\b)([Ii]|[Ss]hould|[Tt]hey|[Ww]e|[Ww]hat|[Ww]ho|[Ww]ould|[Yy]ou)'ve", r"\1\2 have", text)

    return text

def word_negation(text):
    text = re.sub(r"(\b)([Aa]re|[Cc]ould|[Dd]id|[Dd]oes|[Dd]o|[Hh]ad|[Hh]as|[Hh]ave|[Ii]s|[Mm]ight|[Mm]ust|[Ss]hould|[Ww]ere|[Ww]as|[Ww]ould)n[']?t", r"\1\2 not", text)
    text = re.sub(r"(\b)([Cc]a)n't", r"\1\2n not", text)
    text = re.sub(r"(\b)([Ww])on't", r"\1\2ill not", text)
    text = re.sub(r"(\b)([Ss])han't", r"\1\2hall not", text)

    return text

def preprocess_text(text):
    text = phrase_expanding(text)
    text = word_negation(text)
    
    return text

In [7]:
processed_data = qa_data
processed_data['question'] = qa_data.apply(lambda x: preprocess_text(x['question']), axis=1)
processed_data['answer'] = processed_data.apply(lambda x: preprocess_text(x['answer']), axis=1)

<ipython-input-7-d7f9ece6d906>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['question'] = qa_data.apply(lambda x: preprocess_text(x['question']), axis=1)
<ipython-input-7-d7f9ece6d906>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['answer'] = processed_data.apply(lambda x: preprocess_text(x['answer']), axis=1)


In [8]:
processed_data.head(20)

,question,answer
1354503,Why my inside stomach and sides itch feelling ...,See your doctor. I recommend going to your doc...
2181719,Is glaucoma common in young women?,No. Glaucoma is generally a disease of aging. ...
1903046,I have a 2 year old daughter who was diagnosed...,Ventriculomegaly without hydrocephalus is asso...
132377,Can sex cause cold sores?,Yes - oral sex. One of the most common sexuall...
367237,My brain MRI showed multiple T2/flair hyperint...,yes I would consult a neurologist.
1444266,Is there a back surgery where you can cut off ...,Decompressive laminectomies can remove bony co...
2005266,I have had ongoing ovarian cysts that rupture....,No. No clear association.
1720835,Does subungual melanoma have to be a strait l...,Any shape.
2221199,Having extra beats and spells of hr 130-140. M...,Best to discuss with your doctor to see if som...
1989492,Can you inject humira (adalimumab) anywhere on...,"If the shots are painful, massage the skin of ..."


In [9]:
from keras.preprocessing.text import Tokenizer

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(processed_data['question'] + processed_data['answer'])

In [11]:
tokenized_data = processed_data
tokenized_data['question'] = tokenizer.texts_to_sequences(processed_data['question'].values)
tokenized_data['answer'] = tokenizer.texts_to_sequences(processed_data['answer'].values)

<ipython-input-11-dd29cc112e34>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokenized_data['question'] = tokenizer.texts_to_sequences(processed_data['question'].values)
<ipython-input-11-dd29cc112e34>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokenized_data['answer'] = tokenizer.texts_to_sequences(processed_data['answer'].values)


In [12]:
tokenized_data.head()

,question,answer
1354503,"[89, 16, 474, 168, 5, 1614, 834, 28718, 57, 17...","[27, 17, 41, 6, 327, 271, 3, 17, 41]"
2181719,"[4, 1844, 136, 15, 955, 429]","[29, 1844, 4, 507, 1, 115, 7, 2752, 61, 2987, ..."
1903046,"[6, 12, 1, 73, 196, 116, 801, 172, 72, 399, 22...","[21473, 138, 4659, 4, 415, 22, 2968, 21474, 7,..."
132377,"[10, 122, 37, 233, 1040]","[55, 267, 122, 77, 7, 2, 102, 136, 959, 1145, ..."
367237,"[16, 346, 379, 1180, 632, 8138, 8423, 14188, 8...","[55, 6, 49, 304, 1, 714]"


In [13]:
len(tokenizer.word_index)

62573

In [14]:
data = pd.read_csv('../../data/healthtap_qa_20.csv', index_col=0)

In [28]:
data.sample(n=50)

,question,answer
2235078,Can they shorten overgrown toes?,Yes. Excessively long toes can be surgically s...
24195,Does over-exercising cause the same metabolic ...,Not usually . It is important to have enough h...
1841651,Can a fmri give similar information as a spect...,And with better anatomic resolution and no rad...
140951,My pubic bone hurts at 37 weeks. Can they indu...,Normal for pregnant. I do not think so.
1722418,If im getting ingrown toenail surgery today. H...,You probably. Will not need crutches.You may n...
1690008,Can a 25 year old have black spot floaters in ...,Yes. floaters can appear at any age after abou...
1377614,Can ovarian tumor cause a rib fracture?,"Metastatic cancer can involve many areas, incl..."
1964620,A MRI of the brain and spine it's only one sca...,There are rules imposed by Medicare and Medica...
1906466,Does anyone else think it is wrong that women ...,"Who says... ...That a woman ""must"" have children?"
1122566,Can pregnant women use Carteolol?,Carteolol is FDA Pregnancy Category C (Risk ca...


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 289229 entries, 9 to 2287916
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   question  289229 non-null  object
 1   answer    289229 non-null  object
dtypes: object(2)
memory usage: 6.6+ MB


In [21]:
data['question'] = data.apply(lambda x: preprocess_text(x['question']), axis=1)
data['answer'] = data.apply(lambda x: preprocess_text(x['answer']), axis=1)

In [22]:
data.head(50)

,question,answer
9,My friend had redness in her vagina and and it...,Vaginitis? vulvitis? See her doctor.
23,My vagina hurts badly n burns when itching wha...,Vagina burning. Sounds like a 'yeast infection...
33,My wife has an itching right above her clitori...,See dermatologist for answer and treatment.
38,One side of outer vaginal opening very swollen...,Bartholin's cyst. most likely bartholin's glan...
67,"Some sort of eye infection. Red, swollen, yell...",See a Dr. You need antibiotic drops for eye. W...
74,Stinging and swollen stomach 2 months after va...,Probably not. related to your operation. Con...
89,Urgent help needed- I have an extremely sore a...,Since the cause is. Not known - would recommen...
94,Vagina throbs irritating?,You should see your doctor to look for infecti...
96,Vaginal bumps (clusters) in vaginal canal. No ...,Genital warts? Genital warts are the most like...
104,What STD causes a throbbing pain inside vagina?,"Throbbing pain. A throbbing,stabbing pain can ..."


In [25]:
data = data.drop_duplicates(subset=['question'])

In [26]:
data.head(50)

,question,answer
9,My friend had redness in her vagina and and it...,Vaginitis? vulvitis? See her doctor.
23,My vagina hurts badly n burns when itching wha...,Vagina burning. Sounds like a 'yeast infection...
33,My wife has an itching right above her clitori...,See dermatologist for answer and treatment.
38,One side of outer vaginal opening very swollen...,Bartholin's cyst. most likely bartholin's glan...
67,"Some sort of eye infection. Red, swollen, yell...",See a Dr. You need antibiotic drops for eye. W...
74,Stinging and swollen stomach 2 months after va...,Probably not. related to your operation. Con...
89,Urgent help needed- I have an extremely sore a...,Since the cause is. Not known - would recommen...
94,Vagina throbs irritating?,You should see your doctor to look for infecti...
96,Vaginal bumps (clusters) in vaginal canal. No ...,Genital warts? Genital warts are the most like...
104,What STD causes a throbbing pain inside vagina?,"Throbbing pain. A throbbing,stabbing pain can ..."


In [27]:
data.to_csv('../../data/healthtap_qa_20_processed.csv')